# Quantum Algorithm for an Interferometer
 In this notebook, the [QISKit](https://github.com/QISKit) is used to simulate a Mach-Zehnder Interferometer, with the   results being plotted using Matplotlib.
 Notice that the variables names and comments are in english, but the graphs label are in portuguese.

In [ ]:
###Imports
#QISKit for the simulation
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, register
#Numpy for Linear Algebra
import numpy as np
#Pandas for dataframes
import pandas as pd
#Matplotlib for plotting and graphs
import matplotlib.pyplot as plt
#Basic Units for enabling the use of radians in the plot axis
import basic_units as bau
#Enabling the plots appearing inside the notebook
%matplotlib inline

In [ ]:
#Creating a quantum circuit with a single qubit
q = QuantumRegister(1)
c = ClassicalRegister(1)
qc = QuantumCircuit(q, c)

#Creating a array of values for the phase shift of the qubit
phases = list(np.linspace(0, 1, 40) * 4*np.pi)

#Creating a dataframe where the indexes are the phase values, and the columns 
#correspond to the frequency of the result obtained after the measurement.
df = pd.DataFrame(0, index = phases, columns=['0', '1'])

In [ ]:
#Iterate over the phases
for phase in phases:
    #Reset the quantum circuit,
    #i.e. make the qubit go back to the |0> state
    qc.reset(q)
    #Simulating an interferometer effect
    qc.h(q[0])
    qc.u1(phase, q[0])
    qc.h(q[0])
    
    #Measuring the qubit, and repeating the simulation 8000 times
    qc.measure(q, c)
    job_sim = execute(qc, "local_qasm_simulator", shots = 8000)
    sim_result = job_sim.result()
    
    #Update the frequency values for each result
    if '0' in sim_result.get_counts(qc): df.loc[phase, '0'] = sim_result.get_counts(qc)['0']
    if '1' in sim_result.get_counts(qc): df.loc[phase, '1'] = sim_result.get_counts(qc)['1'] 

#Normalizing the frequencies
df = df/8000

In [ ]:
#Plotting the column for |0>, and creating an axes object
ax = df['0'].plot(style = 'o', legend=True, figsize = (10,5), label = r'$|0 \rangle$')
#Plotting the column for |0>
df['1'].plot(style = 'x', legend=True, label = r'$|1 \rangle$')

#Updating the units of x axis to radians
x = [1*bau.radians]
ax.plot(x, 0, xunits=bau.radians)

#Setting up labels and legends
plt.title('Resultados para o interferômetro (8000 experimentos por valor de fase)')
plt.xlabel('Fase aplicada')
plt.ylabel('Frequência do resultado')
#Enabling the grid
plt.grid()

#Positioning the legend box outside the plot's frame
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
